In [4]:
!pip install ipywidgets
!pip install git+https://github.com/facebookresearch/EGG.git

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/facebookresearch/EGG.git to /tmp/pip-req-build-ivll0upe
  Running command git clone -q https://github.com/facebookresearch/EGG.git /tmp/pip-req-build-ivll0upe
  Resolved https://github.com/facebookresearch/EGG.git to commit 18d72d86cf9706e7ad82f94719b56accd288e59a


In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import itertools
import os
import pathlib
import pickle
from functools import reduce
import torch
import torch.nn as nn
import egg.core as core
from egg.zoo.objects_game.util import compute_binomial
from egg.core import language_analysis


from torchvision import datasets, transforms
from torch import nn
from torch.nn import functional as F
from torch.utils import data

import matplotlib.pyplot as plt
import random
import numpy as np
from torch import tensor

from pylab import rcParams
rcParams['figure.figsize'] = 5, 10

from __future__ import print_function

import argparse
import operator
import pathlib

import numpy as np
import torch.nn.functional as F
import torch.utils.data

import egg.core as core
from egg.core.util import move_to
from egg.zoo.objects_game.archs import Receiver, Sender
# from archs import Receiver, Sender
from egg.zoo.objects_game.features import VectorsLoader
from egg.zoo.objects_game.util import (
    compute_baseline_accuracy,
    compute_mi_input_msgs,
    dump_sender_receiver,
    entropy,
    mutual_info,
)

# For convenince and reproducibility, we set some EGG-level command line arguments here
opts = core.init(params=['--random_seed=7', # will initialize numpy, torch, and python RNGs
                         '--lr=1e-3',   # sets the learning rate for the selected optimizer 
                         '--batch_size=32',
                         '--optimizer=adam'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def get_params(params):
    parser = argparse.ArgumentParser()

    input_data = parser.add_mutually_exclusive_group()
    input_data.add_argument(
        "--perceptual_dimensions",
        type=str,
        default="[5, 5, 5, 5, 5]",
        help="Number of features for every perceptual dimension",
    )
    input_data.add_argument(
        "--load_data_path",
        type=str,
        help="Path to .npz data file to load",
    )

    parser.add_argument(
        "--n_distractors",
        type=int,
        default=3,
        help="Number of distractor objects for the receiver (default: 3)",
    )
    parser.add_argument(
        "--train_samples",
        type=float,
        default=128000,
        help="Number of tuples in training data (default: 1e6)",
    )
    parser.add_argument(
        "--validation_samples",
        type=float,
        default=16000,
        help="Number of tuples in validation data (default: 1e4)",
    )
    parser.add_argument(
        "--test_samples",
        type=float,
        default=4000,
        help="Number of tuples in test data (default: 1e3)",
    )
    parser.add_argument(
        "--data_seed",
        type=int,
        default=111,
        help="Seed for random creation of train, validation and test tuples (default: 111)",
    )
    parser.add_argument(
        "--shuffle_train_data",
        action="store_true",
        default=True,
        help="Shuffle train data before every epoch (default: False)",
    )

    parser.add_argument(
        "--sender_hidden",
        type=int,
        default=50,
        help="Size of the hidden layer of Sender (default: 50)",
    )
    parser.add_argument(
        "--receiver_hidden",
        type=int,
        default=50,
        help="Size of the hidden layer of Receiver (default: 50)",
    )

    parser.add_argument(
        "--sender_embedding",
        type=int,
        default=10,
        help="Dimensionality of the embedding hidden layer for Sender (default: 10)",
    )
    parser.add_argument(
        "--receiver_embedding",
        type=int,
        default=10,
        help="Dimensionality of the embedding hidden layer for Receiver (default: 10)",
    )

    parser.add_argument(
        "--sender_cell",
        type=str,
        default="gru",
        help="Type of the cell used for Sender {rnn, gru, lstm} (default: rnn)",
    )
    parser.add_argument(
        "--receiver_cell",
        type=str,
        default="gru",
        help="Type of the cell used for Receiver {rnn, gru, lstm} (default: rnn)",
    )

    parser.add_argument(
        "--sender_lr",
        type=float,
        default=0.001,
        help="Learning rate for Sender's parameters (default: 1e-1)",
    )
    parser.add_argument(
        "--receiver_lr",
        type=float,
        default=0.001,
        help="Learning rate for Receiver's parameters (default: 1e-1)",
    )
    parser.add_argument(
        "--temperature",
        type=float,
        default=1,
        help="GS temperature for the sender (default: 1.0)",
    )
    parser.add_argument(
        "--mode",
        type=str,
        default="gs",
        help="Selects whether Reinforce or GumbelSoftmax relaxation is used for training {gs only at the moment}"
        "(default: rf)",
    )

    parser.add_argument(
        "--output_json",
        action="store_true",
        default=False,
        help="If set, egg will output validation stats in json format (default: False)",
    )

    parser.add_argument(
        "--evaluate",
        action="store_true",
        default=True,
        help="Evaluate trained model on test data",
    )

    parser.add_argument(
        "--dump_data_folder",
        type=str,
        default='./dump_data/',
        help="Folder where file with dumped data will be created",
    )
    parser.add_argument(
        "--dump_msg_folder",
        type=str,
        default=None,
        help="Folder where file with dumped messages will be created",
    )

    parser.add_argument(
        "--debug",
        action="store_true",
        default=False,
        help="Run egg/objects_game with pdb enabled",
    )
    

    args = core.init(parser, params)

    check_args(args)

    return args

def check_args(args):
    args.train_samples, args.validation_samples, args.test_samples = (
        int(args.train_samples),
        int(args.validation_samples),
        int(args.test_samples),
    )

    try:
        args.perceptual_dimensions = eval(args.perceptual_dimensions)
    except SyntaxError:
        print(
            "The format of the # of perceptual dimensions param is not correct. Please change it to string representing a list of int. Correct format: '[int, ..., int]' "
        )
        exit(1)

    if args.debug:
        import pdb

        pdb.set_trace()

    args.n_features = len(args.perceptual_dimensions)
    print(args.perceptual_dimensions)

    # can't set data loading and data dumping at the same time
    assert not (
        args.load_data_path and args.dump_data_folder
    ), "Cannot set folder to dump data while setting path to vectors to be loaded. Are you trying to dump the same vectors that you are loading?"

    args.dump_msg_folder = (
        pathlib.Path(args.dump_msg_folder) if args.dump_msg_folder is not None else None
    )

    if (not args.evaluate) and args.dump_msg_folder:
        print(
            "| WARNING --dump_msg_folder was set without --evaluate. Evaluation will not be performed nor any results will be dumped. Please set --evaluate"
        )


def loss(
    _sender_input, _message, _receiver_input, receiver_output, _labels, _aux_input
):
    acc = (receiver_output.argmax(dim=1) == _labels).detach().float()
    loss = F.cross_entropy(receiver_output, _labels, reduction="none")
    return loss, {"acc": acc}

In [7]:
def main(params, symbol_size, start=True):
    listener_path = './models/listener.pt'
    speaker_path = './models/speaker.pt'
    opts = get_params(params)

    ########################################################
    device = torch.device("cuda")  # Uncomment this
    random.seed(None)
    randseed = random.randrange(1, 10000)
    print('randseed:', randseed)
    ########################################################

    data_loader = VectorsLoader(
        perceptual_dimensions=opts.perceptual_dimensions,
        n_distractors=opts.n_distractors,
        batch_size=opts.batch_size,
        train_samples=opts.train_samples,
        validation_samples=opts.validation_samples,
        test_samples=opts.test_samples,
        shuffle_train_data=opts.shuffle_train_data,
        dump_data_folder=opts.dump_data_folder,
        load_data_path=opts.load_data_path,
        seed=randseed,
    )

    train_data, validation_data, test_data = data_loader.get_iterators()

    data_loader.upd_cl_options(opts)

    if opts.max_len > 1:
        baseline_msg = 'Cannot yet compute "smart" baseline value for messages of length greater than 1'
    else:
        baseline_msg = (
            f"\n| Baselines measures with {opts.n_distractors} distractors and messages of max_len = {opts.max_len}:\n"
            f"| Dummy random baseline: accuracy = {1 / (opts.n_distractors + 1)}\n"
        )
        if -1 not in opts.perceptual_dimensions:
            baseline_msg += f'| "Smart" baseline with perceptual_dimensions {opts.perceptual_dimensions} = {compute_baseline_accuracy(opts.n_distractors, opts.max_len, *opts.perceptual_dimensions)}\n'
        else:
            baseline_msg += f'| Data was loaded froman external file, thus no perceptual_dimension vector was provided, "smart baseline" cannot be computed\n'

    print(baseline_msg)

    # If there is no sender, initialize a new sender
    # Else load an existing state dict into the sender module
    sender = Sender(
        n_features=data_loader.n_features, n_hidden=opts.sender_hidden
    )

    receiver = Receiver(
        n_features=data_loader.n_features, linear_units=opts.receiver_hidden
    )

    if not start:
        print('loading model for speaker')
        sender_dict = load_model(sender, speaker_path)
        sender.load_state_dict(sender_dict)
        print('loading model for listener')
        receiver_dict = load_model(receiver, listener_path)
        receiver.load_state_dict(receiver_dict)

    if opts.mode.lower() == "gs":
        sender = core.RnnSenderGS(
            sender,
            opts.vocab_size,
            opts.sender_embedding,
            opts.sender_hidden,
            cell=opts.sender_cell,
            max_len=symbol_size,
            temperature=opts.temperature,
        )

        receiver = core.RnnReceiverGS(
            receiver,
            opts.vocab_size,
            opts.receiver_embedding,
            opts.receiver_hidden,
            cell=opts.receiver_cell,
        )

        game = core.SenderReceiverRnnGS(sender, receiver, loss)
    else:
        raise NotImplementedError(f"Unknown training mode, {opts.mode}")

    optimizer = torch.optim.AdamW(
        [
            {"params": game.sender.parameters(), "lr": opts.sender_lr},
            {"params": game.receiver.parameters(), "lr": opts.receiver_lr},
        ]
    )
    callbacks = [core.ConsoleLogger(as_json=True), core.InteractionSaver()]
    if opts.mode.lower() == "gs":
        callbacks.append(core.TemperatureUpdater(agent=sender, decay=0.9, minimum=0.1))
    trainer = core.Trainer(
        game=game,
        optimizer=optimizer,
        train_data=train_data,
        validation_data=validation_data,
        callbacks=callbacks,
    )

    trainer.train(n_epochs=opts.n_epochs)
        
    if opts.evaluate:
        is_gs = opts.mode == "gs"
        (
            sender_inputs,
            messages,
            receiver_inputs,
            receiver_outputs,
            labels,
        ) = dump_sender_receiver(
            game, test_data, is_gs, variable_length=True, device=device
        )

        receiver_outputs = move_to(receiver_outputs, device)
        labels = move_to(labels, device)

        receiver_outputs = torch.stack(receiver_outputs)
        labels = torch.stack(labels)

        tensor_accuracy = receiver_outputs.argmax(dim=1) == labels
        accuracy = torch.mean(tensor_accuracy.float()).item()

        unique_dict = {}

        for elem in sender_inputs:
            target = ""
            for dim in elem:
                target += f"{str(int(dim.item()))}-"
            target = target[:-1]
            if target not in unique_dict:
                unique_dict[target] = True

        print(f"| Accuracy on test set: {accuracy}")

        compute_mi_input_msgs(sender_inputs, messages)
        analysis = language_analysis.Disent(False, compute_bosdis=True, compute_posdis=True, vocab_size=vocab_size)
        textfile = open("out.txt", "w")
        for i, msg in enumerate(messages):
            while (len(messages[i]) <= symbol_size):
                messages[i] = torch.hstack((messages[i], tensor(0).to(device)))
            textfile.write(str(messages[i]) + "\n")
        textfile.close()
        topsim = language_analysis.TopographicSimilarity.compute_topsim(torch.stack(sender_inputs).detach().cpu(),
                                                                        torch.stack(messages).detach().cpu(), 'hamming',
                                                                        'hamming')
        bosdis = analysis.bosdis(torch.stack(sender_inputs), torch.stack(messages), opts.vocab_size)
        posdis = analysis.posdis(torch.stack(sender_inputs), torch.stack(messages))
        mi = mutual_info(sender_inputs, messages)
        output_mess = open(f'./msgs/{opts.perceptual_dimensions}.pickle', 'wb')
        pickle.dump({'messages': messages}, output_mess)
        pickle.dump({'objects': sender_inputs}, output_mess)
            
        print(f"entropy sender inputs {entropy(sender_inputs)}")
        print(f"mi sender inputs msgs {mi}")
        print(f"bosdis {bosdis}")
        print(f"posdis {posdis}")
        print(f"topsim {topsim}")

        if opts.dump_msg_folder:
            opts.dump_msg_folder.mkdir(exist_ok=True)
            msg_dict = {}

            output_msg = (
                f"messages_{opts.perceptual_dimensions}_vocab_{opts.vocab_size}"
                f"_maxlen_{symbol_size}_bsize_{opts.batch_size}"
                f"_n_distractors_{opts.n_distractors}_train_size_{opts.train_samples}"
                f"_valid_size_{opts.validation_samples}_test_size_{opts.test_samples}"
                f"_slr_{opts.sender_lr}_rlr_{opts.receiver_lr}_shidden_{opts.sender_hidden}"
                f"_rhidden_{opts.receiver_hidden}_semb_{opts.sender_embedding}"
                f"_remb_{opts.receiver_embedding}_mode_{opts.mode}"
                f"_scell_{opts.sender_cell}_rcell_{opts.receiver_cell}.msg"
            )

            output_file = opts.dump_msg_folder / output_msg
            with open(output_file, "w") as f:
                f.write(f"{opts}\n")
                for (
                        sender_input,
                        message,
                        receiver_input,
                        receiver_output,
                        label,
                ) in zip(
                    sender_inputs, messages, receiver_inputs, receiver_outputs, labels
                ):
                    sender_input = ",".join(map(str, sender_input.tolist()))
                    message = ",".join(map(str, message.tolist()))
                    distractors_list = receiver_input.tolist()
                    receiver_input = "; ".join(
                        [",".join(map(str, elem)) for elem in distractors_list]
                    )
                    if is_gs:
                        receiver_output = receiver_output.argmax()
                    f.write(
                        f"{sender_input} -> {receiver_input} -> {message} -> {receiver_output} (label={label.item()})\n"
                    )

                    if message in msg_dict:
                        msg_dict[message] += 1
                    else:
                        msg_dict[message] = 1

                sorted_msgs = sorted(
                    msg_dict.items(), key=operator.itemgetter(1), reverse=True
                )
                f.write(
                    f"\nUnique target vectors seen by sender: {len(unique_dict.keys())}\n"
                )
                f.write(f"Unique messages produced by sender: {len(msg_dict.keys())}\n")
                f.write(f"Messagses: 'msg' : msg_count: {str(sorted_msgs)}\n")
                f.write(f"\nAccuracy: {accuracy}")

    print('saving listener model as:', listener_path)
    save_model(receiver, listener_path)
    print('saving speaker model as:', speaker_path)
    save_model(sender, speaker_path)
    return accuracy, posdis, bosdis, topsim, mi

In [8]:
def dict2string(d):
    """
    Convert a dict d to string s
    :param d: Dictionary object?
    :return s: a string of all the elements in a dict
    """
    s = []

    for k, v in d.items():
        if type(v) in (int, float):
            s.append(f"--{k}={v}")
        elif type(v) is bool and v:
            s.append(f"--{k}")
        elif type(v) is str:
            assert (
                '"' not in v
            ), f"Key {k} has string value {v} which contains forbidden quotes."
            s.append(f"--{k}={v}")
        else:
            raise Exception(f"Key {k} has value {v} of unsupported type {type(v)}.")
            # print(s)
    return s


def get_dims(features=5, feature_dim=6, random=False):
    """
    Generate a feature vector for n features and k feature dimensions
    :param features: How many features should an object have?
    :param agent_type: How many dimension should a single feature have?
    :return dims: A feature vector of n features X k feature dimension, example: n = 3, k = 5 -> dims: [0, 3, 2]
    """
    dims = [None] * features
    if random:
        for feature in range(0,features):
            dims[feature] = random.randrange(1, feature_dim)
    else:
        dims = [feature_dim] * features
    return dims

In [9]:
def save_model(module, path):
    """
    https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict
    """
    state_dict = module.agent.state_dict()
    torch.save(state_dict, path)
    print('--Saving complete--')
    
    
def load_model(module, path):
    state_dict = torch.load(path)
    return state_dict

In [10]:
# Experiment params
vocab_size = 500
random_seed = 0
n_epoch = 20
batch_size = 64
n_distractors = 3
dims = get_dims(features=3, feature_dim=7)
perceptual_dimensions = str(dims)

In [11]:
random_seed = 0

exp_params = dict(
    vocab_size=vocab_size,
    random_seed=random_seed,
    n_epoch=n_epoch,
    batch_size=batch_size,
    n_distractors=n_distractors,
    perceptual_dimensions=str(dims),
    max_len = 5,
    dump_msg_folder = 'dump',
    train_samples = 5000,
    test_samples = 2000,
    validation_samples = 2000,
    sender_hidden = 300,
    receiver_hidden = 300,
)


In [12]:
def get_paramstring(vocab_size = 500,
               random_seed = 0,
               n_epoch = 50,
               batch_size = 64,
               n_distractors = 9,
               dims = [4, 4]):
    
    dims_constr = get_dims(features=dims[0], feature_dim=dims[1]),
    perceptual_dimensions = str(dims_constr[0])

    exp_params = dict(
        vocab_size=vocab_size,
        random_seed=random_seed,
        n_epoch=n_epoch,
        batch_size=batch_size,
        n_distractors=n_distractors,
        perceptual_dimensions=perceptual_dimensions,
        max_len = 5,
        dump_msg_folder = 'dump',
        train_samples = 4000,
        test_samples = 1000,
        validation_samples = 1000,
        sender_hidden = 300,
        receiver_hidden = 300,
    )
    
    params = dict2string(exp_params)
    return params
    


In [ ]:
import json


def emergent_communication(meaning_space, symbol_size=5, conversation_rounds=10, experiments=30):
    dims = [2, 3, 4, 5, 6, 7, 10, 15, 20, 30]
    for experiment in range(experiments):
        accs = []
        training_accs = {}
        for conversation_round in range(conversation_rounds):
            meaning_space[1] = dims[conversation_round] # Make meaning space larger
            parameters = get_paramstring(dims=meaning_space)
            start = True if conversation_round == 0 else False
            acc, posdis, bosdis, topsim, MI = main(parameters, symbol_size, start=start)
            print(f'accurracy between sender and receiver : {acc}' )

            training_accs[f'{conversation_round}_acc'] = acc
            training_accs[f'{conversation_round}_posdis'] = posdis
            training_accs[f'{conversation_round}_bosdis'] = bosdis
            training_accs[f'{conversation_round}_topsim'] = topsim
            training_accs[f'{conversation_round}_MI'] = MI
            accs.append(acc)
            
        with open(f'accs_2/exp_{experiment+29}.json', 'w') as exp_file:
             exp_file.write(json.dumps(training_accs))
    return accs


meaning_space = [4, 2]
emergent_communication(meaning_space, symbol_size=5, conversation_rounds=10)

[2, 2, 2, 2]
randseed: 67
Cannot yet compute "smart" baseline value for messages of length greater than 1
{"loss": 2.099583148956299, "acc": 0.19062499701976776, "length": 6.0, "mode": "test", "epoch": 1}
{"loss": 1.8266143798828125, "acc": 0.20000000298023224, "length": 6.0, "mode": "test", "epoch": 2}
{"loss": 0.7702405452728271, "acc": 0.609375, "length": 6.0, "mode": "test", "epoch": 3}
{"loss": 0.4007873237133026, "acc": 0.7802083492279053, "length": 6.0, "mode": "test", "epoch": 4}
{"loss": 0.3154848515987396, "acc": 0.8802083134651184, "length": 6.0, "mode": "test", "epoch": 5}
{"loss": 0.2195562869310379, "acc": 0.8770833611488342, "length": 6.0, "mode": "test", "epoch": 6}
{"loss": 0.14935627579689026, "acc": 0.9166666865348816, "length": 6.0, "mode": "test", "epoch": 7}
{"loss": 0.22628207504749298, "acc": 0.8770833611488342, "length": 6.0, "mode": "test", "epoch": 8}
{"loss": 0.13096323609352112, "acc": 0.9197916388511658, "length": 6.0, "mode": "test", "epoch": 9}
{"loss": 